<a href="https://colab.research.google.com/github/markoo26/thehappymountain/blob/main/Pandas_vs_Polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🚚 Imports
import polars as pl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
import plotly.express as px

In [ ]:
#@title ✅ Print Pandas and Polars versions

print(f"Pandas version: {pd.__version__}")
print(f"Polars version: {pl.__version__}")

# 📖 Intro

🐼 The goal of this Colab is to perform benchmark of the so-far most popular Python library for data analysis - **pandas** vs. a new one that is becoming more and more commonly used **polars**.

⏰ **End to end one click run (see below) should take ~ 3 minutes for the default setup**.

📋 Idea is to:

* setup a class `PandasPolarsBenchmark` that gathers different experiments stored in a subclass `PandasPolarsExperiment`, which is supposed to calculate times of execution of equivalent operations.
* build **dummy dataset in two variants**:
 - long (default 10 million of records) - for dumping into CSV
 - short (default 1 million of records) - for dumping into JSON
to eventually get along with the limited resources of the Colab Pro.
* run multiple experiments that should cover exemplary EDA use cases,

📁 Outputs are stored in the `PandasPolarsBenchmark` class, which then offers a method to plot all experiments at once and show final verdict in how many use cases Polars beats Pandas (or reverse) and by how much.

❗ There is a caveat, that **benchmark is performed per experiment in different ways**:

*  running operation once,

or

*  running operation in a loop multiple times

decision here is purely subjective to the author, was made in order to achieve balance between getting conclusive results and being able to produce results in a reasonable time.

🪄 Feel free to **hit Cmd / Ctrl + F9** to run all of the cells at once to reevaluate these two libraries. One cool use case **would be to evaluate different versions of each library** since maybe after time the results of the benchmark could be different.

⚠️ You may be interested in benchmarking different sizes of the long and short DataFrames, bigger than 10 and 1 million. Remember though, to do so, you'd need **this Colab to be run on Colab Pro with high RAM setup (i.e. 50 GBs)**. That being said, if you have less resources available
and the runtime crashes, please go to the [Create a dummy DataFrame](https://colab.research.google.com/drive/1NuWKtpYoQIltnOiMJov22yhGVpUFYgK0#scrollTo=QHVsEKes1ICP&line=1&uniqifier=1) step and downsize the dummy data, so that the runtime's RAM can accomodate it.

🦉 My personal motivation for this Colab was to see on my own how Polars works compared to Pandas as well as to setup some framework to do similar benchmarks in the future.




In [ ]:
#@title 💾 Create a dummy DataFrame and dump it to CSV and JSON for performance tests
#@markdown There are two size params, `long` for CSV and `short` for JSON.
#@markdown Please keep `short` at least 10x smaller than `long`

file_size_long = 10**7 #@param
file_size_short = 10**6 #@param

long_dummy_data = pd.DataFrame(columns = ['person_id', 'age', 'salary', 'experience_in_months', 'rating'])
short_dummy_data = pd.DataFrame(columns = ['person_id', 'age', 'salary', 'experience_in_months', 'rating'])

long_dummy_data['person_id'] = np.random.randint(low=10**6, high = 10**7-1, size=(file_size_long,))
long_dummy_data['age'] = np.random.randint(low=18, high = 65, size=(file_size_long,))
long_dummy_data['salary'] = np.random.randint(low=50, high = 1000, size=(file_size_long,)) * 100
long_dummy_data['experience_in_months'] = np.random.randint(low=1, high = 240, size=(file_size_long,))
long_dummy_data['rating']=np.random.choice(['A', 'B', 'C', 'D', 'E'], size=(file_size_long,))

short_dummy_data['person_id'] = np.random.randint(low=10**6, high = 10**7-1, size=(file_size_short,))
short_dummy_data['age'] = np.random.randint(low=18, high = 65, size=(file_size_short,))
short_dummy_data['salary'] = np.random.randint(low=50, high = 1000, size=(file_size_short,)) * 100
short_dummy_data['experience_in_months'] = np.random.randint(low=1, high = 240, size=(file_size_short,))
short_dummy_data['rating']=np.random.choice(['A', 'B', 'C', 'D', 'E'], size=(file_size_short,))

# Create dummy reference table that indicates which country given person with
# particular person_id is `from`

data_for_joins_pd = long_dummy_data[['person_id']]
data_for_joins_pd['country'] = np.random.choice(['Poland', 'Ukraine', 'USA'], size=(file_size_long,))
data_for_joins_pl = pl.from_pandas(data_for_joins_pd)

# Dump to the CSVs

long_dummy_data.to_csv('dummy_data_long.csv')
short_dummy_data.to_json('dummy_data_long.json')

In [ ]:
#@title 👪 Setup required classes to gather all experiments in one place
#@markdown Created are `PandasPolarsBenchmark` & `PandasPolarsExperiment` classes.
#@markdown This can be reused for any similar experiments in the future.
class PandasPolarsBenchmark():
    def __init__(self):
        self.experiments = []
        self.global_summary = pd.DataFrame(columns=['experiment_name', 'library', 'exec_time'])

    def add_experiment(self, experiment):
        self.experiments.append(experiment)

    def summarize_experiments(self):
        self.global_summary = self.global_summary.iloc[0:0]
        for experiment in self.experiments:
            experiment.calculate_times()
            self.global_summary = pd.concat([self.global_summary, experiment.summary])
        return self.global_summary

    def plot_final_verdict(self):
        return px.bar(benchmark.summarize_experiments(),
                      x='experiment_name',
                      y='exec_time',
                      color='library',
                      barmode = 'group',
                      title=f'Final verdict for Polars vs. Pandas benchmark: {self.get_final_result()}')

    def get_final_result(self):

        colnames = ['experiment_name', 'better_library']

        pivot_df = self.summarize_experiments() \
                    .pivot_table(index='experiment_name',
                                    columns='library',
                                    values='exec_time',
                                    aggfunc='min')

        pivot_df['better_library'] = pivot_df.apply(lambda row: 'pandas' \
                                                    if row['pandas'] < row['polars'] \
                                                    else 'polars',
                                                    axis=1)


        raw_score = pivot_df.reset_index()[colnames] \
                            .groupby(by='better_library') \
                            .count() \
                            .reset_index()\
                            .values

        score = f"""{raw_score[1][0].capitalize()}: {raw_score[1][1]} -- {raw_score[0][0].capitalize()}: {raw_score[0][1]} """

        return score

class PandasPolarsExperiment():
    def __init__(self, experiment_name):
        self.experiment_name = experiment_name
        self.pl_start_time = None
        self.pl_end_time = None
        self.pd_start_time = None
        self.pd_end_time = None
        self.fig = None
        self.summary = pd.DataFrame(columns=['experiment_name', 'library', 'exec_time'])

    def calculate_times(self):
        self.polars_exec_time = round(self.pl_end_time - self.pl_start_time,2)
        self.pandas_exec_time = round(self.pd_end_time - self.pd_start_time,2)

# A little bit hacky way, but the goal was to not use .append method
# which is to be deprecated soon

    def summarize_experiment(self):
        self.summary = pd.concat([self.summary,
        pd.DataFrame({
                'experiment_name': [self.experiment_name],
                'library': ['polars'],
                'exec_time': [self.polars_exec_time]
            })], ignore_index = True)
        self.summary = pd.concat([self.summary,
        pd.DataFrame({
                'experiment_name': [self.experiment_name],
                'library': ['pandas'],
                'exec_time': [self.pandas_exec_time]
            })], ignore_index = True)

    def plot_experiment(self):
        self.fig = px.bar(experiment.summary, x='library', y = 'exec_time', color='library', title=f'{experiment.experiment_name}')
        self.fig.show()

benchmark = PandasPolarsBenchmark()

# ⚔️ Perform the benchmarking

Create a dummy data first with `person_id | age | salary | experience_in_months | rating `

In [ ]:
#@title [Experiment 1️⃣] - Reading JSON file to a DataFrame

experiment = PandasPolarsExperiment('JSON import')

# Pandas

experiment.pd_start_time = time.time()
pandas_df = pd.read_json('dummy_data_long.json')
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
polars_df = pl.read_json('dummy_data_long.json')
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)



In [ ]:
#@title [Experiment 2️⃣] - Reading CSV file to a DataFrame

experiment = PandasPolarsExperiment('CSV import')

# Pandas

experiment.pd_start_time = time.time()
pandas_df = pd.read_csv('dummy_data_long.csv')
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
polars_df = pl.read_csv('dummy_data_long.csv')
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)



In [ ]:
#@title [Experiment 3️⃣] - Single Mean aggregation

experiment = PandasPolarsExperiment('Simple Groupby')

# Pandas

experiment.pd_start_time = time.time()
pd_groupby = pandas_df[["salary", "rating"]].groupby(by='rating').mean()
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
pl_groupby = polars_df \
                .select(["salary", "rating"]) \
                .group_by('rating') \
                .mean()
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()

benchmark.add_experiment(experiment)

In [ ]:
#@title [Experiment 4️⃣] - Sampling 20% of the data

experiment = PandasPolarsExperiment('Sampling of 20% of the data')

# Pandas

experiment.pd_start_time = time.time()
sampled_pandas_df = pandas_df.sample(frac=0.2)
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
sampled_pl_df = polars_df.sample(fraction=0.2)
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)

In [ ]:
#@title [Experiment 5️⃣] - Pivoting (rows -> columns)

experiment = PandasPolarsExperiment('Pivotting & aggregating the data')

# Pandas

experiment.pd_start_time = time.time()
pivotted_pd_df = pandas_df.pivot_table(values="salary", index="age", columns="rating", aggfunc=np.mean)
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
pivotted_pl_df = polars_df.pivot(values="salary", index="age", columns="rating", aggregate_function='mean')
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)


In [ ]:
#@title [Experiment 6️⃣] - Melting (columns -> rows)

# Preparation

pivotted_pd_df_with_reset_index = pivotted_pd_df.reset_index()
experiment = PandasPolarsExperiment('Melting simple df 10 000x times')

# Pandas

experiment.pd_start_time = time.time()
for _ in range(10000):
    melted_pd_df = pivotted_pd_df_with_reset_index.melt(id_vars=['age'])
melted_pd_df = pivotted_pd_df_with_reset_index.melt(id_vars=['age'])
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
for _ in range(10000):
    melted_pl_df = pivotted_pl_df.melt(id_vars=['age'])
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)

In [ ]:
#@title [Experiment 7️⃣] - Iterate through the whole dataframe

experiment = PandasPolarsExperiment('Iterate through the 10 milion rows from a Dataframe')

# Pandas

experiment.pd_start_time = time.time()
for i in pandas_df.head(1000000).iterrows():
    pass
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
for i in polars_df.head(1000000).rows():
    pass
experiment.pl_end_time = time.time()

# Wrap-up
experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)

In [ ]:
#@title [Experiment 8️⃣] - Joins

experiment = PandasPolarsExperiment('Perform a LEFT JOIN of DF with 10m records with reference table')

# Pandas

experiment.pd_start_time = time.time()
output_df_pd = pandas_df.merge(data_for_joins_pd.head(1000000), how='left')
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
output_df_pl = polars_df.join(data_for_joins_pl.head(1000000),
                              how='left', left_on='person_id',
                              right_on='person_id')
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)

In [ ]:
#@title [Experiment 9️⃣] - Map / lambda

experiment = PandasPolarsExperiment('Add a calculated field')

# Pandas

experiment.pd_start_time = time.time()
pandas_df['age_bucket'] = pandas_df['age'].apply(lambda x: int(x/10) )
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
polars_df = polars_df.with_columns(((pl.col("age")/ 10).cast(pl.Int32)).alias("age_bucket"))
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)

In [ ]:
#@title [Experiment 🔟] - Sorting

experiment = PandasPolarsExperiment('Sorting the DataFrame')

# Pandas

experiment.pd_start_time = time.time()
pandas_df = pandas_df.sort_values(by=['person_id'])
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
polars_df = polars_df.sort(by=['person_id'])
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)

In [ ]:
#@title [Experiment 1️⃣1️⃣] - Filter using sophisticated conditions

experiment = PandasPolarsExperiment('Complex filtering')

# Pandas

experiment.pd_start_time = time.time()
filtered_pd_df = pandas_df[(pandas_df['age'] > 40) &
                           (pandas_df['salary'] < 50000) &
                           (pandas_df['rating'].isin(['A', 'B']))]
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
filtered_pl_df = polars_df.filter(
                        (pl.col("age") > 40) &
                        (pl.col("salary") < 50000) &
                        (pl.col("rating").is_in(['A', 'B'])))
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)


In [ ]:
#@title [Experiment 1️⃣2️⃣] - Dump back to CSV

experiment = PandasPolarsExperiment('Dumping back a DataFrame to CSV')

# Pandas

experiment.pd_start_time = time.time()
pandas_df.to_csv('pd_dataframe.csv')
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
polars_df.write_csv('pl_dataframe.csv')
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)


In [ ]:
#@title [Experiment 1️⃣3️⃣] - Unique values

experiment = PandasPolarsExperiment('Get unique values from each column into a dict')

pd_uniques, pl_uniques = {}, {}

# Pandas

experiment.pd_start_time = time.time()
for column in pandas_df.columns:
    pd_uniques[column] = pandas_df[column].unique().tolist()
experiment.pd_end_time = time.time()
experiment.pl_start_time = time.time()

# Polars

for column in polars_df.columns:
    pl_uniques[column] = polars_df[column].unique().to_list()
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)


In [ ]:
#@title [Experiment 1️⃣4️⃣] - Convert datatypes

#@markdown Perform 100 times the conversion to int16 and back to int64

experiment = PandasPolarsExperiment('Convert 100 times to int16 and back to int64')
pd_uniques, pl_uniques = {}, {}

# Pandas

experiment.pd_start_time = time.time()
for _ in range(100):
    pandas_df['age'] = pandas_df['age'].astype('int16')
    pandas_df['age'] = pandas_df['age'].astype('int64')
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
for _ in range(100):
    polars_df = polars_df.with_columns(pl.col("age").cast(pl.Int16), name="age")
    polars_df = polars_df.with_columns(pl.col("age").cast(pl.Int64), name="age")
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)



In [ ]:
#@title [Experiment 1️⃣5️⃣] - Copy a DataFrame

experiment = PandasPolarsExperiment('Copy a DataFrame')

# Pandas

experiment.pd_start_time = time.time()
dummy_pandas_df = pandas_df.copy()
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
dummy_polars_df = polars_df.clone()
experiment.pl_end_time = time.time()

# Wrap-up

experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()
benchmark.add_experiment(experiment)



In [ ]:
#@title [Experiment 1️⃣6️⃣] - Concatenate pd.DataFrames

experiment = PandasPolarsExperiment('Get first million records and concat it')

# Pandas

experiment.pd_start_time = time.time()
dummy_pandas_df = dummy_pandas_df.head(1000000)
for _ in range(10):
    dummy_pandas_df = pd.concat([dummy_pandas_df, dummy_pandas_df.head(1000000)])
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
dummy_polars_df = dummy_polars_df.head(1000000)
for _ in range(10):
    dummy_polars_df = pl.concat([dummy_polars_df, dummy_polars_df.head(1000000)])

# Wrap-up

experiment.pl_end_time = time.time()
experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()

benchmark.add_experiment(experiment)



In [ ]:
#@title [Experiment 1️⃣7️⃣] - Dropping Nulls

# Force each second record have NaN in Age field

# Preparation

nullified_pd_df = pandas_df.copy()
nullified_pd_df['age'] = nullified_pd_df['age'].apply(lambda x: np.nan if x % 2 == 0 else x)


nullified_pl_df = polars_df.clone()
nullified_pl_df = nullified_pl_df.with_columns(
    pl.when(pl.col("age") % 2 == 0)
    .then(None)  # Polars uses None for null values
    .otherwise(pl.col("age"))
    .alias("age")
)

experiment = PandasPolarsExperiment('Drop NULLs from the DataFrame')

# Pandas

experiment.pd_start_time = time.time()
nullified_pd_df = nullified_pd_df.dropna()
experiment.pd_end_time = time.time()

# Polars

experiment.pl_start_time = time.time()
nullified_pl_df = nullified_pl_df.drop_nulls()
experiment.pl_end_time = time.time()

# Summary
experiment.calculate_times()
experiment.summarize_experiment()
experiment.plot_experiment()

benchmark.add_experiment(experiment)


#  🛬 Final verdict

In [ ]:
#@title 📊 Plot a combined bar plot & calculate final score
benchmark.plot_final_verdict()